<a href="https://colab.research.google.com/github/mrsamsonn/fbprophet-yfinance-next_hour_prediction/blob/main/git_fb_prophet_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!pip install pystan~=2.14
!pip install prophet

In [7]:
%pip install yfinance

In [8]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
from prophet import Prophet

In [9]:
#date
past = (date.today()-timedelta(days=1))
print("date:", past)

date: 2023-09-16


In [10]:
data = yf.download('BTC-USD', start=past, interval='5m')
data.reset_index(inplace=True)
#utc to pdt timezone
data['Datetime'] = pd.to_datetime(data['Datetime']) - timedelta(hours=7)
data['Datetime'] = data['Datetime'].dt.strftime('%Y/%m/%d %H:%M:%S')

[*********************100%%**********************]  1 of 1 completed


In [12]:
data.reset_index(inplace=True)
data = data[['Datetime','Close']]
data.columns = ['ds','y']
data

,ds,y
0,2023/09/15 17:00:00,26606.925781
1,2023/09/15 17:05:00,26628.158203
2,2023/09/15 17:10:00,26629.873047
3,2023/09/15 17:15:00,26629.667969
4,2023/09/15 17:20:00,26656.228516
...,...,...
259,2023/09/16 14:35:00,26575.505859
260,2023/09/16 14:40:00,26591.183594
261,2023/09/16 14:45:00,26582.869141
262,2023/09/16 17:00:00,26569.785156


In [13]:
prophet = Prophet(daily_seasonality=False)
prophet.add_seasonality(name='hourly', period=0.0416, fourier_order=1)
prophet.fit(data)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpi5w3c43n/eqmn255e.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpi5w3c43n/2klserj2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67211', 'data', 'file=/tmp/tmpi5w3c43n/eqmn255e.json', 'init=/tmp/tmpi5w3c43n/2klserj2.json', 'output', 'file=/tmp/tmpi5w3c43n/prophet_modelu9j8882j/prophet_model-20230917000955.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:09:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:09:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [14]:
future_dates = prophet.make_future_dataframe(periods=1)
predictions = prophet.predict(future_dates)

In [15]:
from prophet.plot import plot_plotly

plot_plotly(prophet, predictions)